In [1]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import numpy as np
import os
import matplotlib.pyplot as plt 
from datetime import date
import pandas as pd
import numpy as np
import cv2


cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml") 
cap=cv2.VideoCapture(0)

x_train=[]
y_train=[]
for f in os.listdir('dataset/train'):
    for filename in os.listdir('dataset/train/'+f):
        img=load_img('dataset/train/'+f+'/'+filename)
        img=img.resize( (100,100))
        img=img_to_array(img)
        x_train.append(img)
        y_train.append(f)



x_train=np.array(x_train).reshape(-1,30000)
y_train=np.array(y_train)
x_train,y_train=shuffle(x_train,y_train)

model = KNeighborsClassifier(n_neighbors=10)
model.fit(x_train, y_train)





present_student=set()
while True:
    ret,frame=cap.read()
    if not ret:
        continue
    faces=cascade.detectMultiScale(frame, 1.3, 4)
    faces=sorted(faces,key=lambda k :k[2]*k[3])
    for (x,y,w,h) in faces[-1:]:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+offset+w]
        face_section=cv2.resize(face_section,(100,100),interpolation = cv2.INTER_AREA)
        face=np.array(face_section).reshape(-1,30000)
        prediction=model.predict(face)
        
        # face section is (100,100,3) size image  use ur model to make prediction 
        # prediction = model.predict()   omprakash make prediction using ur model
        # now store ur prediction in set i.e present_student
        present_student.add(prediction[0])
        
        frame = cv2.putText(frame, prediction[0], (50, 50) , cv2.FONT_HERSHEY_SIMPLEX , 1,(255,0,0), 2, cv2.LINE_AA) 
    cv2.imshow("frame1",frame)    
    if cv2.waitKey(1) & 0xFF == ord('q'):     # press q to stop taking attendance
        break
cap.release()    
cv2.destroyAllWindows()     



present_student=np.array(list(present_student))
present_student=present_student.reshape(present_student.shape[0],-1)
df = pd.DataFrame (present_student)
# here  storing present_student data in excel file
today = date.today()                    
print("Today's date:", today)
file_path='attendance/'+today.strftime("%d-%m-%Y")+'.xlsx'
df.to_excel(file_path, index=False,header=['roll no'])

print('attendance taken successfully')


Using TensorFlow backend.


Today's date: 2020-04-14
attendance taken successfully
